### FAIR VALUE PPP CALCULATOR
Authors: Debborah Lucas, Jorge J. Montesinos

In [36]:
## Install packages
# install.packages("tidyverse")
# install.packages("optiRum")
library(tidyverse)
library(tidyr)
library(dplyr)
library(readr)
library("ggplot2")
library("optiRum")
library("purrr")

In [ ]:
# Load data (Hypothetical case):
df <- read_csv("C:/Users/Montesinos/BH my notebooks/BH r_model (MIT)/BH_p3_calculator/BH_data_hyp_case_v2.csv")
df

### INPUTS 

In [9]:
## INPUTS 
# Project's generals:
n  <- 40            # total project duration (years)
n1 <- 5             # construction period (years)
n2 <- n-n1          # operation period (years)
rf <- 0.02          # risk-free rate (%)
inflation <- 0.021  # expected inflation
vol1 <- 0.30        # constant volatility
vol2 <- rep(c(0,0.5,0.4,0.3,0.2), c(6,2,2,2,29))   # varying volatility
vol3 <- tibble(vol1,vol2) # volatility dataframe

# Calculate cost of capital (ra):
Beta <- mean(c(0.80,0.81))             # asset beta: average btw transportation (.8) and trucking (.81)
premium <- 0.06                        # equity premium
ra <- rf + Beta*(premium) - inflation  # real fair value discount rate (%)
ra <- round(ra, digits = 3)


# Other inputs:
strike <- 15000000    # strike price (minimum guaranteed revenue)
durations <- c(5,10,20)


### BLACKS'S MODEL CALCULATIONS:

In [42]:
#### Net cash flows (net_cfs):
df <- df %>%
  mutate(net_cfs = revs - capital - o_m) %>%
  # Calculate Revenue Forward Prices (fw):
  mutate(fw = case_when(time < 13 ~ revs*exp(rf*time)/(1+ra)^time, 
                        time >= 13 ~ revs[13]*exp(rf*time)/(1+ra)^time)) %>%
  
  # Calculate Put value (put):
  # insert d1 and d2
  mutate(d1 = (log(fw/strike) + time*(vol1^2)/2) / (vol1*sqrt(time))) %>%
  mutate(d1 = replace(d1, d1 == -Inf, NA)) %>%
  mutate(d2 = d1-(vol1*sqrt(time))) %>%
  # insert N(-d1) and N(-d2)     
  mutate(N_d1 = pnorm(-d1), 
         N_d2 = pnorm(-d2)) %>%
  # insert put value (revenue guarantee) for each time t     
  mutate(put = exp(-rf*time)*(strike*N_d2 - fw*N_d1))
df
tail(df)

time,revs,capital,o_m,net_cfs,fw,d1,d2,N_d1,N_d2,put
0,0,0.0e+00,0,0,0,NA,NA,NA,NA,NA
1,0,1.2e+08,0,-120000000,0,NA,NA,NA,NA,NA
2,0,1.2e+08,0,-120000000,0,NA,NA,NA,NA,NA
3,0,1.2e+08,0,-120000000,0,NA,NA,NA,NA,NA
4,0,1.2e+08,0,-120000000,0,NA,NA,NA,NA,NA
5,0,1.2e+08,0,-120000000,0,NA,NA,NA,NA,NA
6,44689868,0.0e+00,15186757,29503111,38251134,1.641326,0.90647861,0.05036493,0.1823413,717167.1
7,45901851,0.0e+00,15950734,29951117,38282885,1.577312,0.78358677,0.05736186,0.2166413,915992.6
8,47044260,0.0e+00,16714711,30329549,38231409,1.526888,0.67835960,0.06339447,0.2487719,1114534.3
9,49329077,0.0e+00,17044000,32285077,39062125,1.513448,0.61344791,0.06508296,0.2697901,1256724.9


time,revs,capital,o_m,net_cfs,fw,d1,d2,N_d1,N_d2,put
35,73008389,0,21788000,51220389,26146670,1.200497,-0.5743265,0.1149731,0.7171266,3848901
36,73008389,0,21788000,51220389,25477428,1.194301,-0.6056985,0.1161800,0.7276425,3871953
37,73008389,0,21788000,51220389,24825317,1.188503,-0.6363261,0.1173177,0.7377181,3890063
38,73008389,0,21788000,51220389,24189896,1.183073,-0.6662516,0.1183902,0.7473748,3903507
39,73008389,0,21788000,51220389,23570740,1.177986,-0.6955135,0.1194011,0.7566332,3912553
40,73008389,0,21788000,51220389,22967431,1.173219,-0.7241474,0.1203540,0.7655124,3917459


In [ ]:
# Value of Minimum Revenue Guarantees under different durations:
mrg_put   <- map_dbl(durations, ~sum(df$put[7:(6 +.x)]))
mrg_put

map2(durations, mrg_put, ~print(.x, .y))

In [29]:
## TIFIA LOAN:
# Inputs:
princ_tf <- 120000000     # principal TIFIA ($)
rate_tf  <- 0.0225        # rate TIFIA (%)
rate_mkt <- rate_tf + rf  # market rate (%)
n_tf     <- 30            # maturity TIFIA (years)
start_tf <- 5             # start year TIFIA

# Calculations
pmt =(rate_tf*princ_tf)/(1-(1+rate_tf)^(-n_tf))  # payment (TIFIA)
pv_mkt = -PV(rate_mkt,n_tf,-pmt)                 # PV(debt payments) @market rate

# The present value of the credit subsidy would be the difference between:
#   1) PV(debt payments) discounted at subsidized borrowing rate (@2.25%)
#   2) PV(debt payments) discounted at estimated market rate (@4.25)
pv_subsidy <- (princ_tf+pv_mkt)/(1+rate_mkt)^start_tf

# As percentage of TIFIA's principal (at year 5):
subsidy_perc <- (princ_tf+pv_mkt)/princ_tf

# Print results:
cat("pmt = ", pmt,"\n")
cat("pv_mkt = ", pv_mkt, "\n")
cat("pv_subsidy = ", pv_subsidy, "\n")
cat("subsidy_perc = ", subsidy_perc,"\n")


pmt =  5543921 
pv_mkt =  -93021547 
pv_subsidy =  21909715 
subsidy_perc =  0.2248204 


In [30]:
## MUNI:
# Inputs:
princ_muni <- 230000000     # principal MUNI ($)
rate_muni  <- 0.0475        # rate MUNI (%)
tax_muni   <- 0.2           # tax rate MUNI (%)
n_muni     <- 40            # maturity MUNI (years)

# Calculations
pv_muni = -PV(rate_muni,n_muni,-(tax_muni*rate_muni*princ_muni))                 # PV(debt payments) @market rate

# Print result:
cat("pv_muni = ", pv_muni,"\n")

pv_muni =  -38812190 
